In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE95425"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE95425"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE95425.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE95425.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE95425.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cell-type specific gene programs of the normal human nephron define kidney cancer subtypes"
!Series_summary	"Comprehensive transcriptome studies of cancers often rely on corresponding normal tissue samples to serve as a transcriptional reference. In this study we performed in-depth analyses of normal kidney tissue transcriptomes from TCGA and demonstrate that the histological variability in cellularity, inherent in the kidney architecture, lead to considerable transcriptional differences between samples. This should be considered when comparing expression profiles of normal and cancerous kidney tissues. We exploited these differences to define renal cell-specific gene signatures and used these as framework to analyze renal cell carcinoma (RCC) ontogeny. Chromophobe RCCs express FOXI1-driven genes that define collecting duct intercalated cells whereas HNF-regulated genes, specific for proximal tubule cells, are an integral part of clear cell and pa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable

# Gene Expression Availability
# Based on the Series_summary, this study is about transcriptomes of normal kidney tissue,
# which suggests gene expression data is available
is_gene_available = True

# Data Availability and Type Conversion
# For trait (Kidney_Chromophobe), we can use the sampling depth as a proxy (row 2)
# It seems the samples are from different depths of kidney tissue (cortex, cortex/medulla, medulla)
trait_row = 2

# No explicit age information is available
age_row = None

# No explicit gender information is available
gender_row = None

# Conversion functions
def convert_trait(value):
    """
    Convert sampling depth information to a binary value:
    1 for cortex (since we're studying kidney chromophobe which is related to cortex),
    0 for medulla, and 0.5 for mixed cortex/medulla
    """
    if not value or ':' not in value:
        return None
        
    value = value.split(':', 1)[1].strip().lower()
    
    if 'cortex' in value and 'medulla' in value:
        return 0.5  # Mixed
    elif 'cortex' in value:
        return 1  # Cortex
    elif 'medulla' in value:
        return 0  # Medulla
    else:
        return None

# Since age and gender are not available, we don't need to define their conversion functions
convert_age = None
convert_gender = None

# Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, 'clinical_data.csv')
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(clinical_features)
        print("Clinical features preview:", preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 27367 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228',
       'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651236',
       'ILMN_1651238', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259',
       'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278',
       'ILMN_1651281', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Let's analyze the gene identifiers shown in the output
# The identifiers starting with "ILMN_" indicate these are Illumina probe IDs,
# not standard human gene symbols. Illumina is a microarray platform, and these
# IDs need to be mapped to standard gene symbols.

# Illumina probe IDs (like ILMN_1343291) need to be converted to gene symbols
# for proper biological interpretation and cross-platform compatibility

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1498611 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify mapping columns
# From the output, we can see:
# - The gene expression data uses 'ILMN_' IDs as identifiers (e.g., ILMN_1343291)
# - The gene annotation data has an 'ID' column containing these same identifiers
# - The 'Symbol' column in the annotation contains gene symbols

# 2. Extract mapping between probe IDs and gene symbols
print("Creating gene mapping from probe IDs to gene symbols...")
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Created mapping with {len(gene_mapping)} entries")

# Show a sample of the mapping
print("\nSample of gene mapping (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the mapping to convert probe-level measurements to gene expression
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted to gene expression data with {len(gene_data)} genes")

# Verify the result with a preview
print("\nPreview of gene expression data (first 5 rows):")
print(preview_df(gene_data.head()))

# Check for data integrity
non_null_count = gene_data.count().sum()
total_elements = gene_data.size
null_percentage = 100 - (non_null_count / total_elements * 100)
print(f"\nNull values: {null_percentage:.2f}% of the data")

# Normalize gene symbols to standard format
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data)} unique genes")

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nSaved gene expression data to {out_gene_data_file}")


Creating gene mapping from probe IDs to gene symbols...
Created mapping with 44837 entries

Sample of gene mapping (first 5 rows):
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

Converting probe-level measurements to gene expression data...
Converted to gene expression data with 17999 genes

Preview of gene expression data (first 5 rows):
{'GSM2510512': [872.3328899999999, 11.0, 22.0, 3095.599, 1837.8201], 'GSM2510513': [529.13506, 11.0, 22.0, 1993.715, 2131.865], 'GSM2510514': [55.676279, 11.0, 22.0, 183.4023, 2454.2156], 'GSM2510515': [52.654965000000004, 11.0, 22.0, 193.08295, 1946.8962], 'GSM2510516': [832.20962, 11.0, 22.0, 3110.6108, 2073.256], 'GSM2510517': [1360.64394, 11.0, 22.0, 1384.135, 2864.6042], 'GSM2510518': [1368.9527165, 11.0, 22.0, 1532.8215, 2951.128], 'GSM2510519': [15

After normalization: 17405 unique genes



Saved gene expression data to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE95425.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Extract clinical features first since we didn't do this properly in Step 2
print("\nExtracting clinical features...")
try:
    # Get the trait row, which was identified in step 2
    trait_row = 2
    
    # Define convert_trait function as it was defined in step 2
    def convert_trait(value):
        """
        Convert sampling depth information to a binary value:
        1 for cortex (since we're studying kidney chromophobe which is related to cortex),
        0 for medulla, and 0.5 for mixed cortex/medulla
        """
        if not value or ':' not in value:
            return None
            
        value = value.split(':', 1)[1].strip().lower()
        
        if 'cortex' in value and 'medulla' in value:
            return 0.5  # Mixed
        elif 'cortex' in value:
            return 1  # Cortex
        elif 'medulla' in value:
            return 0  # Medulla
        else:
            return None
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,
        convert_age=None,
        gender_row=None,
        convert_gender=None
    )
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical features: {e}")
    is_trait_available = False
    clinical_df = None

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if clinical_df is not None and not gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("First few clinical sample columns:", list(clinical_df.columns)[:5])
        print("First few genetic sample columns:", list(gene_data.columns)[:5])
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 3. Handle missing values systematically
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 4. Determine whether the trait and demographic features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("Cannot link data: clinical or genetic data is missing")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() else pd.DataFrame(),
    note="Kidney tissue samples with different sampling depths (cortex/medulla) used to study kidney chromophobe"
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Extracting clinical features...
Clinical data saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE95425.csv

Linking clinical and genetic data...
First few clinical sample columns: ['GSM2510512', 'GSM2510513', 'GSM2510514', 'GSM2510515', 'GSM2510516']
First few genetic sample columns: ['GSM2510512', 'GSM2510513', 'GSM2510514', 'GSM2510515', 'GSM2510516']
Linked data shape: (53, 17406)

Handling missing values...


After handling missing values, data shape: (53, 17406)

Checking for bias in features...
Quartiles for 'Kidney_Chromophobe':
  25%: 0.0
  50% (Median): 0.5
  75%: 1.0
Min: 0.0
Max: 1.0
The distribution of the feature 'Kidney_Chromophobe' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Kidney_Chromophobe/GSE95425.csv
